In [ ]:
!wandb login 55d09ffedb7a5b9c08dfddc17d834220dbf0bfa4

In [ ]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping

import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint


In [ ]:
# Initialize W&B project (done once)
wandb.init(project="wildfire-prediction-experiments")
wandb.finish()

In [ ]:
def display_random_sample(dataset_dir):
    classes = [
        class_name for class_name in os.listdir(dataset_dir)
        if os.path.isdir(os.path.join(dataset_dir, class_name))
    ]
    plt.figure(figsize=(15, 10))
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(dataset_dir, class_name)
        image_files = [
            file_name for file_name in os.listdir(class_dir)
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        
        if not image_files:
            continue

        random_image = random.choice(image_files)
        img_path = os.path.join(class_dir, random_image)
        img = load_img(img_path)
        
        plt.subplot(1, len(classes), i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{class_name} ({img.size[0]}x{img.size[1]})")
    plt.show()


In [ ]:
train_dir = 'fire_data/train'
valid_dir = 'fire_data/valid'
test_dir = 'fire_data/test'

print("Training set samples:")
display_random_sample(train_dir)

print("Validation set samples:")
display_random_sample(valid_dir)

print("Test set samples:")
display_random_sample(test_dir)

In [ ]:
def load_data(directory, img_size=(32, 32)):
    X, Y = [], []
    for direct in os.listdir(directory):
        direct_path = os.path.join(directory, direct)
        if not os.path.isdir(direct_path):
            continue
        for filename in os.listdir(direct_path):
            img_path = os.path.join(direct_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            img = img.astype(np.float32) / 255.0
            X.append(img)
            Y.append(direct)
    return np.array(X), np.array(Y)

x_train, y_train = load_data(train_dir)
x_val, y_val = load_data(valid_dir)
x_test, y_test = load_data(test_dir)

# Convert labels to binary
y_train = np.array([1 if label == 'wildfire' else 0 for label in y_train])
y_val = np.array([1 if label == 'wildfire' else 0 for label in y_val])
y_test = np.array([1 if label == 'wildfire' else 0 for label in y_test])

print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

In [ ]:
def build_model(input_shape, conv_layers, conv_filters, dense_units, dropout_rate, activation, optimizer, learning_rate):
    model = Sequential()
    model.add(Input(shape=input_shape))
    
    for i in range(conv_layers):
        model.add(Conv2D(conv_filters[i], (3, 3), padding='same', activation=activation))
        model.add(MaxPooling2D((2, 2)))
        model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(dense_units, activation=activation))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Optimizer not recognized")

    model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Generate 50-100 experiments by systematically varying hyperparameters

experiment_configs = []

# Define parameter grids
conv_layer_options = [1, 2]                 # 2 options
dense_units_options = [64, 128, 256]        # 3 options
dropout_options = [0.2, 0.3]                # 2 options
learning_rates = [1e-3, 1e-4]               # 2 options
optimizers = ['adam', 'sgd', 'rmsprop']     # 3 options

# Total combinations: 2 (conv_layers) * 3 (dense_units) * 2 (dropout) * 2 (lr) * 3 (optim) = 72 experiments

for conv_layers in conv_layer_options:
    for dense_units in dense_units_options:
        for dropout_rate in dropout_options:
            for lr in learning_rates:
                for optimizer in optimizers:
                    exp = {
                        "conv_layers": conv_layers,
                        "conv_filters": [32]*conv_layers,  # Use 32 filters for each layer as a baseline
                        "dense_units": dense_units,
                        "dropout_rate": dropout_rate,
                        "activation": "relu",
                        "optimizer": optimizer,
                        "learning_rate": lr,
                        "batch_size": 32,
                        "epochs": 5  # Keep epochs small for demonstration
                    }
                    experiment_configs.append(exp)

len(experiment_configs)

In [ ]:
# Run the experiments
for i, config in enumerate(experiment_configs):
    print(f"Starting experiment {i+1}/{len(experiment_configs)} with config: {config}")
    wandb.init(project="wildfire-prediction-experiments", config=config, reinit=True)
    
    model = build_model(
        input_shape=(32,32,3),
        conv_layers=config["conv_layers"],
        conv_filters=config["conv_filters"],
        dense_units=config["dense_units"],
        dropout_rate=config["dropout_rate"],
        activation=config["activation"],
        optimizer=config["optimizer"],
        learning_rate=config["learning_rate"]
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    
    model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=config["batch_size"],
        epochs=config["epochs"],
        callbacks=[
            WandbMetricsLogger(),
            WandbModelCheckpoint(filepath="model.keras"),
            early_stopping
        ],
        verbose=1
    )
    
    # Evaluate on test data
    test_predictions = model.predict(x_test)
    y_pred = (test_predictions > 0.5).astype("int32").ravel()

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')

    wandb.log({
        "test_accuracy": accuracy,
        "test_precision": precision,
        "test_recall": recall
    })

    wandb.finish()

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
